In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def corr2d(X, K):  #@save
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [5]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [6]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
Y = corr2d(X, K)
Y

tensor([[6., 2., 0., 0., 0., 4., 6.],
        [6., 2., 0., 0., 0., 4., 6.],
        [6., 2., 0., 0., 0., 4., 6.],
        [6., 2., 0., 0., 0., 4., 6.],
        [6., 2., 0., 0., 0., 4., 6.]])

In [8]:
corr2d(X.t(), K)

tensor([[6., 6., 6., 6., 6.],
        [1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [5., 5., 5., 5., 5.],
        [6., 6., 6., 6., 6.]])

In [18]:
# 1) Build X with a vertical edge (6x8), per D2L
X2d = torch.ones((6, 8))
X2d[:, 2:] = 0  # left side 1s, right side 0s → vertical edge between cols 1 and 2

# 2) 1x2 kernel (detects horizontal differences)
K = torch.tensor([[1., -1.]])

# 3) Target Y via cross-correlation (6x7)
Y2d = corr2d(X2d, K)

# 4) Reshape to NCHW for Conv2d
X = X2d.reshape(1, 1, 6, 8)
Y = Y2d.reshape(1, 1, 6, 7)

# 5) Lazy conv layer: 1 out channel, kernel (1,2), no bias
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

lr = 3e-2
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # gradient descent on the kernel elements (weights)
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

# Optional: see learned kernel (should approach [[1, -1]] up to scale)


epoch 2, loss 6.276
epoch 4, loss 3.470
epoch 6, loss 1.920
epoch 8, loss 1.063
epoch 10, loss 0.588


In [19]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.7703, -0.6284]])